In [1]:
from VSM import *
from FeatureExtraction import *

In [2]:
# num_components = min(tfidf_matrix_uc.shape[0], tfidf_matrix_code.shape[1])
# num_components = min(num_components, 100)
# LSA_model = TruncatedSVD(n_components=num_components)

# # if train_or_test == "train":
# LSA_data_useCases = LSA_model.fit_transform(tfidf_matrix_uc)
# LSA_data_codes = LSA_model.fit_transform(tfidf_matrix_code) 
# # else:
# #     LSA_data_useCases = LSA_model.transform(tfidf_matrix_uc)
# #     LSA_data_codes = LSA_model.transform(tfidf_matrix_code) 

# LSA_similraity_matrix = cosine_similarity(LSA_data_useCases, LSA_data_codes)
# print(LSA_similraity_matrix)

[[ 0.18482886  0.3273808   0.32296982 ... -0.19929168  0.27819572
   0.35131266]
 [ 0.28823275  0.5802744   0.51701597 ... -0.05965957  0.34313524
   0.33720379]
 [ 0.40728073  0.32797859  0.20740612 ... -0.08974246  0.28103445
   0.26552303]
 ...
 [ 0.46312831  0.22129758  0.10572698 ... -0.13914677  0.30564828
   0.34829635]
 [ 0.21368115  0.39048124  0.33017151 ... -0.16889377  0.54142573
   0.41258289]
 [ 0.52801245  0.07769887 -0.01366914 ...  0.17987461  0.35918657
   0.64894778]]


In [3]:
# test = FeatureExtraction()
# test.LSA(tfidf_matrix_uc, tfidf_matrix_code, "train")

array([[ 0.18482879,  0.32737807,  0.32296327, ..., -0.19928636,
         0.27819406,  0.35130898],
       [ 0.28822455,  0.58027843,  0.51700034, ..., -0.05965165,
         0.34314602,  0.33719957],
       [ 0.40729472,  0.32797612,  0.20740683, ..., -0.08973778,
         0.28103775,  0.26551924],
       ...,
       [ 0.46314434,  0.22129739,  0.10572649, ..., -0.13913957,
         0.30565419,  0.34829004],
       [ 0.2137009 ,  0.39046935,  0.33019448, ..., -0.16890304,
         0.54142993,  0.41255777],
       [ 0.52803358,  0.07769631, -0.01366185, ...,  0.17987321,
         0.35919171,  0.64893806]])

In [28]:
#usecases (100*usecases_nu)
#code (100*codes_nu)
#LSA_similraity_matrix (usecases_nu*codes_nu)
#d1,d2,d3,d4
#u1,u2,u3,u4
#u1*d1,u1*d2,u1*d3 ...
#d1*u1,d1*u2,d1*u3 ...
#each row coresspond between the use cases and each document
# [[d1*c1 d1*c2],[d2*c1 d2*c2]]

# LSA_data_codes=np.array([[1,2,3,4,5] , [6,7,8,9,10]])
# LSA_data_useCases=np.array([[11,12,13,14,15] , [16,17,18,19,21],[2,5,9,11,42]])

# LSA_data_codes_norm=np.repeat(np.linalg.norm(LSA_data_codes,axis=1),LSA_data_useCases.shape[0])
# LSA_data_useCases_norm=np.repeat(np.linalg.norm(LSA_data_useCases,axis=1),LSA_data_codes.shape[0])

# LSA_data_codes_norm=LSA_data_codes_norm.reshape(LSA_data_codes.shape[0],LSA_data_useCases.shape[0]).T 
# LSA_data_useCases_norm=LSA_data_useCases_norm.reshape(LSA_data_useCases.shape[0],LSA_data_codes.shape[0])

# print(LSA_data_codes_norm)
# print(LSA_data_useCases_norm)

# LSA_similraity_matrix=cosine_similarity(LSA_data_useCases, LSA_data_codes)

# print(LSA_similraity_matrix)

[[ 0.18482137  0.3273921   0.32295519 ... -0.19927008  0.27819332
   0.35132647]
 [ 0.28824105  0.58026967  0.5170505  ... -0.05965737  0.34313247
   0.33729178]
 [ 0.40726529  0.32798009  0.20740789 ... -0.08973642  0.28103006
   0.26557118]
 ...
 [ 0.46310731  0.22130393  0.10572647 ... -0.13913685  0.30565015
   0.34834786]
 [ 0.21364449  0.39048035  0.33014368 ... -0.16890047  0.54140078
   0.41261981]
 [ 0.52799292  0.07769306 -0.01365563 ...  0.17986966  0.35917951
   0.64910979]]


## Jensen-Shannon

In [2]:
test = FeatureExtraction()
test.JensenShannon(UCTokens, UC_documents, code_documents)

array([[0.60628914, 0.56214859, 0.62365091, ..., 0.63699536, 0.56847883,
        0.59296509],
       [0.60919967, 0.56296403, 0.59642528, ..., 0.63554238, 0.54885985,
        0.56170336],
       [0.61025351, 0.57647443, 0.61537687, ..., 0.65734326, 0.57873848,
        0.57834334],
       ...,
       [0.56991067, 0.58071035, 0.66233178, ..., 0.61751555, 0.58643133,
        0.58627933],
       [0.5710618 , 0.54848827, 0.6307035 , ..., 0.62615112, 0.55630119,
        0.53115382],
       [0.60452273, 0.54866734, 0.63787855, ..., 0.63255515, 0.5579383 ,
        0.60818104]])

In [7]:
%pip install translate

Note: you may need to restart the kernel to use updated packages.


In [17]:
from translate import Translator

translator = Translator(to_lang='en', from_lang='it')  # Translate to German
translation = translator.translate('Punto Di Ristoro')
print(translation)  # Output: 'Guten Morgen!'

Feeding station
